In [1]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
pd.set_option('display.max_columns',100)

DATA_PATH = '../okko/orig_data'
PREPARED_PATH = './prepared_data/'

In [3]:
data = pd.read_pickle(PREPARED_PATH+'some_wm.pkl')

In [4]:
data.head(40)

rel_dur  type  rating action_enc cons_enc  \
user_uid element_uid                                               
0        6127         0.107821   1.0     NaN      watch        S   
         8168         0.025000   1.0     NaN      watch        S   
         1284         0.022222   1.0     NaN      watch        S   
         1535         0.015385   1.0     NaN      watch        S   
         460          0.033333   1.0     NaN      watch        S   
         7190         0.946515   1.0     NaN      watch        S   
         8183         0.360333   1.0     NaN      watch        S   
         1983         0.858571   1.0     NaN      watch        S   
         6272         0.841667   1.0     NaN      watch        S   
         1016         1.020000   1.0     NaN      watch        S   
         1165         0.510000   1.0     NaN      watch        S   
         5035         0.957037   1.0     NaN      watch        S   
         434          0.893030   1.0     NaN      watch        S   
         6961         1.001042   1.0     NaN      watch        S   
         10150        0.985556   1.0     NaN      watch        S   
         3651         1.222000   1.0     NaN      watch        S   
         2081         0.802222   1.0     NaN      watch        S   
         2429         0.001436   3.0     NaN      watch        S   
         4290         0.425417   1.0     NaN      watch        S   
         2570         0.890185   1.0     NaN      watch        S   
         2143         0.801833   1.0     NaN      watch        S   
         4972         0.000000   1.0     NaN      watch        S   
         944          0.067037   1.0     NaN      watch        S   
         5366         1.453333   1.0     NaN      watch        S   
         2651         0.918333   1.0     NaN      watch        S   
         3567         0.861500   1.0     NaN      watch        S   
         5039         0.512037   1.0     NaN      watch        S   
         3722         0.961500   1.0     NaN      watch        S   
         9175         0.250833   1.0     NaN      watch        S   
         3496         0.200417   1.0     NaN      watch        S   
         3029         1.026458   1.0     NaN      watch        S   
         8747         1.292593   1.0     NaN      watch        S   
         7869         0.928939   1.0     NaN      watch        S   
         2639         0.015476   1.0     NaN      watch        S   
         1550         0.741333   1.0     NaN      watch        S   
         7498         0.281000   1.0     NaN      watch        S   
         2382         0.968194   1.0     NaN      watch        S   
         4935         0.962000   1.0     NaN      watch        S   
         2487         0.980000   1.0     NaN      watch        S   
         7179         0.916190   1.0     NaN      watch        S   

                          first_ts    first_ts_1       diff_ts  duration  \
user_uid element_uid                                                       
0        6127         4.332506e+07           NaN  12339.563912       130   
         8168         4.333740e+07  4.332506e+07     35.834058        80   
         1284         4.333743e+07  4.333740e+07     33.798077        90   
         1535         4.333747e+07  4.333743e+07     30.798518       130   
         460          4.333750e+07  4.333747e+07    120.224309       120   
         7190         4.333762e+07  4.333750e+07   1252.552669       110   
         8183         4.333887e+07  4.333762e+07  15727.796149       100   
         1983         4.335460e+07  4.333887e+07  10453.541197       140   
         6272         4.336505e+07  4.335460e+07    908.506874       100   
         1016         4.336596e+07  4.336505e+07  14180.229511       110   
         1165         4.338014e+07  4.336596e+07     55.607472       110   
         5035         4.338020e+07  4.338014e+07    893.051582        90   
         434          4.338109e+07  4.338020e+07  12655.099645       110   
         6961         4.33

In [ ]:
data_catalogue = pd.read_pickle(PREPARED_PATH+'data_catalogue.pkl')
with open(PREPARED_PATH+'catalogue_features.pkl','rb') as f:
    match_element_row,match_row_element,match_columns,element_matrix = pickle.load(f)

In [ ]:
# Попробуем честно прорешать задачу классификации для оценок, чтобы восстановить зависимости
import lightgbm
import xgboost
import catboost

In [ ]:
data_class = data.loc[~data['rating'].isnull()]

In [ ]:
data_class.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 438790 entries, (1, 1653) to (593486, 7233)
Data columns (total 25 columns):
rel_dur              438790 non-null float64
type                 438790 non-null float64
rating               438790 non-null float64
action_enc           438790 non-null object
cons_enc             357343 non-null object
first_ts             438790 non-null float64
first_ts_1           438790 non-null float64
diff_ts              424176 non-null float64
duration             438790 non-null int64
feature_1            438790 non-null float64
feature_2            438790 non-null float64
feature_3            438790 non-null int64
feature_4            438790 non-null float64
feature_5            438790 non-null float64
is_history           438790 non-null int32
is_future            438790 non-null int32
diff_novice_ts       438790 non-null float64
target               438790 non-null int32
mean_rating_user     438790 non-null float64
num_watched_user     438790 no

In [ ]:
y = data_class['rating'].astype(int)

In [ ]:
data.action_enc.value_counts()

watch                  9047668
bookmate                678865
watch+rate              330622
watch+bookmate          238001
rate                     76818
watch+rate+bookmate      26721
rate+bookmate             4629
Name: action_enc, dtype: int64

In [ ]:
need_cols = ['rel_dur','type','action_enc','cons_enc','diff_ts','feature_1','feature_2','feature_3','feature_4','feature_5',
            'is_history','diff_novice_ts','probably_kp_rate','num_watched_movie']
#'mean_rating_user','mean_target_movie',
data_cut = data_class[need_cols]
data_cut['action_enc'] = data_cut['action_enc'].map({'watch':1,'bookmate':2,'watch+rate':3,'watch+bookmate':4,
                                                    'rate':5,'watch+rate+bookmate':6,'rate+bookmate':7})
data_cut['cons_enc'] = data_cut['cons_enc'].map({'S':1,'P':2,'R':3,})

C:\Users\yabom\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\yabom\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [ ]:
lgb_dataset = lightgbm.Dataset(data_cut,y,categorical_feature=['feature_3','type','action_enc','cons_enc','is_history'],
                              free_raw_data=False)

In [ ]:
from sklearn.metrics import roc_auc_score,accuracy_score

In [ ]:
def binary_error(preds, train_data):
    labels = train_data.get_label()
    lab = pd.get_dummies(labels)
    return 'roc_error', roc_auc_score(lab,preds.reshape((-1,11))) , True


In [ ]:
params = {
    'objective':'softmax',
    'num_class':11,
    'boosting':'dart',
    #'metric':['multi_error'],
}
p = lightgbm.cv(params,lgb_dataset,num_boost_round=10000,early_stopping_rounds=20,verbose_eval=True,feval = binary_error)

C:\Users\yabom\Anaconda3\lib\site-packages\lightgbm\basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[1]	cv_agg's multi_logloss: 2.30092 + 0.00016112	cv_agg's roc_error: 0.499513 + 0.000817307
[2]	cv_agg's multi_logloss: 2.22518 + 0.000187772	cv_agg's roc_error: 0.499577 + 0.000979328
[3]	cv_agg's multi_logloss: 2.16312 + 0.000246754	cv_agg's roc_error: 0.499641 + 0.000957761
[4]	cv_agg's multi_logloss: 2.11088 + 0.00032173	cv_agg's roc_error: 0.499562 + 0.00104792
[5]	cv_agg's multi_logloss: 2.06614 + 0.000365501	cv_agg's roc_error: 0.49957 + 0.00105049
[6]	cv_agg's multi_logloss: 2.02722 + 0.000390226	cv_agg's roc_error: 0.499508 + 0.00107242
[7]	cv_agg's multi_logloss: 1.99311 + 0.00046965	cv_agg's roc_error: 0.499502 + 0.00107765
[8]	cv_agg's multi_logloss: 2.01086 + 0.000437135	cv_agg's roc_error: 0.499501 + 0.00106637
[9]	cv_agg's multi_logloss: 1.97854 + 0.000436572	cv_agg's roc_error: 0.499428 + 0.00110748
[10]	cv_agg's multi_logloss: 1.95 + 0.000518333	cv_agg's roc_error: 0.499372 + 0.00113214


In [ ]:
def binary_error(preds, train_data):
    labels = train_data.get_label()
    
    return 'roc_error', accuracy_score(labels,preds>0.5),True#roc_auc_score(labels,preds) , True#False

In [ ]:
y = 1*(data_class['rating'].astype(int) > 4)
lgb_dataset = lightgbm.Dataset(data_cut,y,categorical_feature=['feature_3','type','action_enc','cons_enc','is_history'],
                              free_raw_data=False)
params = {
    'objective':'binary',
    'boosting':'dart',
    'metric':['auc'],
}
p = lightgbm.cv(params,lgb_dataset,num_boost_round=10000,early_stopping_rounds=20,verbose_eval=True,feval=binary_error)

In [ ]:
model = lightgbm.train(params,lgb_dataset,num_boost_round=110,feval = binary_error)

In [ ]:
lightgbm.plot_importance(model)

In [ ]:
other = data.loc[data.rating.isnull(),need_cols]
other['action_enc'] = other['action_enc'].map({'watch':1,'bookmate':2,'watch+rate':3,'watch+bookmate':4,
                                                    'rate':5,'watch+rate+bookmate':6,'rate+bookmate':7})
other['cons_enc'] = other['cons_enc'].map({'S':1,'P':2,'R':3,})

# lgb_other = lightgbm.Dataset(other,reference=lgb_dataset,categorical_feature=['feature_3','type','action_enc','cons_enc','is_history'],
#                               free_raw_data=False)

In [ ]:
pred = model.predict(other)

In [ ]:
plt.hist(pred - (1-pred))

In [ ]:
uv = pred - (1-pred)
uv_med = np.median(uv)

In [ ]:
uv_med

In [ ]:
# Всем, у кого уверенность больше медианы - добавим 1 и -1 а потом сунем в предикты
data.loc[other[(uv>uv_med) & (pred>0.5)].index,'target_new'] = 1
data.loc[other[(uv>uv_med) & (pred<=0.5)].index,'target_new'] = -1
data.loc[data.rating>4,'target_new'] = 1
data.loc[data.rating<=4,'target_new'] = -1

In [ ]:
data.target_new.value_counts(normalize = True)